In [9]:
from ultralytics import YOLO
import numpy as np

# Load a model
modelpose = YOLO('yolov8n-pose.pt') 

In [10]:
import os

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Thirty videos worth of data
no_sequences = 46

# Videos are going to be 30 frames in length
sequence_length = 15

In [17]:
# Define path to the main folder containing class subfolders
data_path = "MP_Data15"  # Replace with your actual path

# Define actions based on subfolder names (modify as needed)
actions = np.array(os.listdir(data_path))  # Get list of subfolders
print(actions)


['running' 'standing' 'sitting' 'boxing' 'jumping' 'walking' 'squat']


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(15,34)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [21]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [22]:
model.load_weights('action93.h5')

In [ ]:
import cv2 

colors = [(245,117,16), (117,245,16), (16,117,245), (126, 249, 255), (255, 166, 255),(16,117,245), (40, 166, 133)]
def prob_viz(res, actions, input_frame, colors, keypoints):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    top_left_x = int(keypoints[0][:, 0].min().item())  # Minimum x-coordinate across all keypoints
    top_left_y = int(keypoints[0][:, 1].min().item())-20  # Minimum y-coordinate across all keypoints
    bottom_right_x = int(keypoints[0][:, 0].max().item())  # Maximum x-coordinate across all keypoints
    bottom_right_y = int(keypoints[0][:, 1].max().item())+20 # Maximum y-coordinate across all keypoints

    # Define bounding box thickness and color
    thickness = 2
    color = (0, 255, 0)  # Green for bounding box

    # Draw the rectangle
    cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), color, thickness)

    return output_frame


def draw_bounding_box(frame, keypoints):
  # Assuming keypoints[0] contains all keypoints for a single person
  # Identify keypoints for top left and bottom right corners (adjust based on your model)
  top_left_x = int(keypoints[0][:, 0].min().item())  # Minimum x-coordinate across all keypoints
  top_left_y = int(keypoints[0][:, 1].min().item())-20  # Minimum y-coordinate across all keypoints
  bottom_right_x = int(keypoints[0][:, 0].max().item())  # Maximum x-coordinate across all keypoints
  bottom_right_y = int(keypoints[0][:, 1].max().item())+20 # Maximum y-coordinate across all keypoints

  # Define bounding box thickness and color
  thickness = 2
  color = (0, 255, 0)  # Green for bounding box

  # Draw the rectangle
  cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), color, thickness)

  return frame


In [ ]:
# 1. New detection variables
import cv2
sequence = []
sentence = []
threshold = 0.6

cap = cv2.VideoCapture("Dataset/jumping/6032918-hd_1920_1080_25fps.mp4")
# Set mediapipe model 

while cap.isOpened():

    # Read feed
    ret, frame = cap.read()

    if not ret:
        print("No frames left in video. Exiting...")
        break
    
    image = frame

    # Make detections
    results = modelpose.predict(frame)
    # Flatten keypoints
    keypoints = np.array(results[0][0].keypoints.xyn).flatten()

    sequence.append(keypoints)
    sequence = sequence[-15:]
    
    if len(sequence) == 15:
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        print(actions[np.argmax(res)])
        
        
    #3. Viz logic
        if res[np.argmax(res)] > threshold: 
            if len(sentence) > 0: 
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5: 
            sentence = sentence[-5:]

        # Viz probabilities
        image = prob_viz(res, actions, image, colors)
        
    cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
    cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Show to screen
    cv2.imshow('OpenCV Feed', image)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 181.0ms
Speed: 4.0ms preprocess, 181.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 86.5ms
Speed: 8.2ms preprocess, 86.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 467.3ms
Speed: 3.9ms preprocess, 467.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 317.8ms
Speed: 24.6ms preprocess, 317.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 61.1ms
Speed: 12.7ms preprocess, 61.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 488.1ms
Speed: 50.3ms preprocess, 488.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 127.3ms
Speed: 41.5ms preprocess, 127.3ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 54.6ms
Speed: 37.2ms preprocess, 54.6ms inference, 0.8ms postprocess per image at 

1/1 [==============================] - 0s 33ms/step



0: 384x640 1 person, 90.9ms
Speed: 16.9ms preprocess, 90.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


squat
1/1 [==============================] - 0s 42ms/step



0: 384x640 1 person, 89.0ms
Speed: 3.1ms preprocess, 89.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


...

1/1 [==============================] - 0s 31ms/step



0: 384x640 1 person, 127.7ms
Speed: 3.0ms preprocess, 127.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


squat
1/1 [==============================] - 0s 28ms/step



0: 384x640 1 person, 108.5ms
Speed: 6.4ms preprocess, 108.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


squat
1/1 [==============================] - 0s 35ms/step
squat


In [ ]:
# 1. New detection variables
import cv2
sequence = []
sentence = []
threshold = 0.6

cap = cv2.VideoCapture("Dataset/jumping/6032918-hd_1920_1080_25fps.mp4")
# Set mediapipe model 

while cap.isOpened():

    # Read feed
    ret, frame = cap.read()

    if not ret:
        print("No frames left in video. Exiting...")
        break
    
    image = frame

    # Make detections
    results = modelpose.predict(frame)
    # Flatten keypoints
    keypoints = np.array(results[0][0].keypoints.xyn).flatten()

    sequence.append(keypoints)
    sequence = sequence[-15:]
    
    if len(sequence) == 15:
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        print(actions[np.argmax(res)])
        
        
    #3. Viz logic
        if res[np.argmax(res)] > threshold: 
            if len(sentence) > 0: 
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5: 
            sentence = sentence[-5:]

        # Viz probabilities
        image = prob_viz(res, actions, image, colors)
        
    cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
    cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Show to screen
    cv2.imshow('OpenCV Feed', image)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()